In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range

from guruTester import GuruTeplster,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [2]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [17]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

9.999999999998899e-05

In [18]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsFeature(df)
    df = ATRFeature(df)
    df = KeltnerChannelsFeature(df)
    df = RSIFeature(df)
    df = MACDFeature(df)
    df['pivots_l'] = False
    df['pivots_h'] = False

    df['low_perv'] = df.low.shift(1)
    df['high_perv'] = df.high.shift(1)
    df['low_next'] = df.low.shift(-1)
    df['high_next'] = df.high.shift(-1)
    df['pivots_l'] = (df.low_perv > df.low) & (df.low_next > df.low)
    df['pivots_h'] = (df.high_perv < df.high) & (df.high_next < df.high)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    df.pivots_l = df.pivots_l.astype(float)
    df.pivots_h = df.pivots_h.astype(float)
    return df

In [19]:
df_an = featureExtraction(df_an)

In [20]:
# candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
#        'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
#        'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
# df_an = apply_patterns_in_range(df_an)
# for cp in candle_patterns:
#     df_an[cp] = df_an[cp].astype(float)
    
# df_an.head()

In [21]:
# df_an = BollingerBandsFeature(df_an)
# df_an = ATRFeature(df_an)
# df_an = KeltnerChannelsFeature(df_an)
# df_an = RSIFeature(df_an)
# df_an = MACDFeature(df_an)
# df_an

In [22]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

9e-05

In [23]:
# df_an['pivots_l'] = False
# df_an['pivots_h'] = False

# df_an['low_perv'] = df_an.low.shift(1)
# df_an['high_perv'] = df_an.high.shift(1)
# df_an['low_next'] = df_an.low.shift(-1)
# df_an['high_next'] = df_an.high.shift(-1)
# df_an['pivots_l'] = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
# df_an['pivots_h'] = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
# df_an.dropna(inplace=True)
# df_an.reset_index(drop=True, inplace=True)
df_an

,time,open,high,low,close,full_range,body_lower,body_upper,body_bottom_perc,body_top_perc,...,HIST,Feat_MACD,Feat_SIGNAL_MACD,Feat_HIST,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next
0,2020-01-07 23:00:00,1.31169,1.31206,1.31125,1.31195,0.00081,1.31169,1.31195,0.543210,0.864198,...,-0.000163,-0.000984,-0.000821,-0.000163,0.0,0.0,1.31155,1.31222,1.31043,1.31249
1,2020-01-08 00:00:00,1.31244,1.31249,1.31043,1.31138,0.00206,1.31138,1.31244,0.461165,0.975728,...,-0.000159,-0.001020,-0.000861,-0.000159,0.0,1.0,1.31125,1.31206,1.31039,1.31184
2,2020-01-08 01:00:00,1.31139,1.31184,1.31039,1.31125,0.00145,1.31125,1.31139,0.593103,0.689655,...,-0.000149,-0.001048,-0.000898,-0.000149,1.0,0.0,1.31043,1.31249,1.31057,1.31190
3,2020-01-08 02:00:00,1.31126,1.31190,1.31057,1.31108,0.00133,1.31108,1.31126,0.383459,0.518797,...,-0.000138,-0.001071,-0.000933,-0.000138,0.0,0.0,1.31039,1.31184,1.31100,1.31202
4,2020-01-08 03:00:00,1.31109,1.31202,1.31100,1.31161,0.00102,1.31109,1.31161,0.088235,0.598039,...,-0.000081,-0.001034,-0.000953,-0.000081,0.0,0.0,1.31057,1.31190,1.31158,1.31389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19957,2023-03-27 07:00:00,1.22444,1.22465,1.22340,1.22377,0.00125,1.22377,1.22444,0.296000,0.832000,...,0.000363,-0.000547,-0.000910,0.000363,0.0,0.0,1.22399,1.22511,1.22327,1.22396
19958,2023-03-27 08:00:00,1.22377,1.22396,1.22327,1.22363,0.00069,1.22363,1.22377,0.521739,0.724638,...,0.000319,-0.000511,-0.000830,0.000319,0.0,0.0,1.22340,1.22465,1.22188,1.22464
19959,2023-03-27 09:00:00,1.22365,1.22464,1.22188,1.22453,0.00276,1.22365,1.22453,0.641304,0.960145,...,0.000340,-0.000406,-0.000745,0.000340,1.0,0.0,1.22327,1.22396,1.22263,1.22593
19960,2023-03-27 10:00:00,1.22457,1.22593,1.22263,1.22342,0.00330,1.22342,1.22457,0.239394,0.587879,...,0.000271,-0.000407,-0.000678,0.000271,0.0,0.0,1.22188,1.22464,1.22325,1.22658


In [24]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0
SPREAD_IN_POSITON = 0# SPREAD # 0
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = BUY
                    break
        
        df_an['SL'][i] = df_an.low[i-1] 
        df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]
                    
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = SELL
                    break
                    
        df_an['SL'][i] = df_an.high[i-1] + SPREAD_IN_POSITON
        df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] - SPREAD_IN_POSITON) + df_an.close[i]
                    

C:\Users\meysam\AppData\Local\Temp\ipykernel_7076\1697842815.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['SL'][i] = df_an.high[i-1] + SPREAD_IN_POSITON
C:\Users\meysam\AppData\Local\Temp\ipykernel_7076\1697842815.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] - SPREAD_IN_POSITON) + df_an.close[i]
C:\Users\meysam\AppData\Local\Temp\ipykernel_7076\1697842815.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [25]:
X = []
Y = [] 
feature_col = [
#     'open', 'high', 'low', 'close',
#                'RSI_14', 'ATR_14',
#     'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
#     'MACD', 'SIGNAL_MACD', 'HIST',
    
    'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c',
    'Feat_EMA_c', 'Feat_KeUp_c', 'Feat_KeLo_c',
    
    'Feat_BB_MA_o', 'Feat_BB_UP_o', 'Feat_BB_LW_o',
    'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 
    
    'Feat_BB_MA_l', 'Feat_BB_UP_l', 'Feat_BB_LW_l',
    'Feat_EMA_l', 'Feat_KeUp_l', 'Feat_KeLo_l',
    
    'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
    'Feat_EMA_h', 'Feat_KeUp_h', 'Feat_KeLo_h',
    
    'Feat_ATR_14',
    'Feat_gains','Feat_wins_rma', 'Feat_losses_rma', 'Feat_RSI_14', 
#     'Feat_MACD', 'Feat_SIGNAL_MACD', 'Feat_HIST',
    
        'full_range',# this is added ...
       'body_lower', 'body_upper', 
        'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
    'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [26]:

indeces = []
for i in range(4,len(df_an)):
    if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        feature.append(df_an.pivots_h[i-2])
        feature.append(df_an.pivots_l[i-2])
        feature.append(df_an.pivots_h[i-3])
        feature.append(df_an.pivots_l[i-3])
        feature.append(df_an.pivots_h[i-4])
        feature.append(df_an.pivots_l[i-4])
        for fc in feature_col:
            feature.append(df_an[fc][i])
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [27]:
SPLIT_INDEX = int(0.8*len(X))
X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]


In [28]:
def softmax(x):
#     return np.exp(x ) / np.exp(x.sum())
    return(np.exp(x - np.max(x)) / np.exp(x - np.max(x)).sum())

def relu(x):
    return (abs(x) + x) / 2


In [29]:
coefs_0 = np.genfromtxt('coefs_0.csv', delimiter=',')
coefs_1 = np.genfromtxt('coefs_1.csv', delimiter=',')
coefs_2 = np.genfromtxt('coefs_2.csv', delimiter=',')
intercepts_0 = np.genfromtxt('intercepts_0.csv', delimiter=',')
intercepts_1 = np.genfromtxt('intercepts_1.csv', delimiter=',')
intercepts_2 = np.genfromtxt('intercepts_2.csv', delimiter=',')

In [30]:
l1 = np.matmul(X_test,coefs_0) + intercepts_0
l1 = relu(l1)
l2 = np.matmul(l1, coefs_1) + intercepts_1
l2 = relu(l2)
l3 = np.matmul(l2, coefs_2) + intercepts_2
l3 = softmax(l3)
pred_test = np.argmax(l3,axis=1)-1
l3.shape


(1729, 3)

In [31]:
(df_an[df_an.SIGNAL !=0].TP ==0).sum()

0

In [32]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()


In [33]:
(df_an[df_an.SIGNAL !=0].SL ==0).sum()

0

In [34]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
# y_perdict = clf.predict(X_test)
for i,idx in enumerate(indeces_test):
    if i > 0:
        if df_test.pivots_h[idx-1] or df_test.pivots_l[idx-1] :
            df_test.SIGNAL.loc[idx] = pred_test[i]
#             if(pred_test[i] == 1):
#                 df_test['SL'][idx] = df_an.low[idx-1] 
#                 df_test['TP'][idx] = 2*(df_an.close[idx] - df_an.low[idx-1] ) + df_an.close[idx]
#             if(pred_test[i] == -1):
#                 df_test['SL'][idx] = df_an.high[idx-1] 
#                 df_test['TP'][idx] = 2*(df_an.close[idx] - df_an.high[idx-1] ) + df_an.close[idx]
# df_test.reset_index(drop=True, inplace=True)


C:\Users\meysam\AppData\Local\Temp\ipykernel_7076\3189138399.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.SIGNAL.loc[idx] = pred_test[i]


In [35]:
(df_test[df_test.SIGNAL !=0].SL ==0).sum()

0

In [36]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51532 entries, 0 to 51531
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    51532 non-null  datetime64[ns]
 1   open    51532 non-null  float64       
 2   high    51532 non-null  float64       
 3   low     51532 non-null  float64       
 4   close   51532 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 2.0 MB


In [37]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

prepare_data...
run_test...
Result:
win rate:  0.3629842180774749
-1.0    444
 2.0    253
Name: result, dtype: int64


In [ ]:
mt5.initialize()

In [ ]:
to_date = datetime.now()
from_date= datetime(2022,8,3,0,0)
rates = mt5.copy_rates_range(pair, mt5.TIMEFRAME_H1,from_date,to_date)
df_an2 = pd.DataFrame(rates)
df_an2["time"] = pd.to_datetime(df_an2["time"], unit="s")
df_an2.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)


In [ ]:
df_an2 = featureExtraction(df_an2)


In [ ]:
def getFeatureVec(df):
    Xfea_vec = []
    indeces2 = []
    for i in range(4,len(df)):
    #     if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
        feature = []
        feature.append(df.pivots_h[i-1])
        feature.append(df.pivots_l[i-1])
        feature.append(df.pivots_h[i-2])
        feature.append(df.pivots_l[i-2])
        feature.append(df.pivots_h[i-3])
        feature.append(df.pivots_l[i-3])
        feature.append(df.pivots_h[i-4])
        feature.append(df.pivots_l[i-4])
        for fc in feature_col:
            feature.append(df[fc][i])
            feature.append(df[fc][i-1])
            feature.append(df[fc][i-2])
            feature.append(df[fc][i-3])
            feature.append(df[fc][i-4])

        Xfea_vec.append(feature)
    #         Y.append(df_an.SIGNAL[i])
        indeces2.append(i)

    Xfea_vec = np.array(Xfea_vec)
    return Xfea_vec

In [ ]:
X2 = getFeatureVec(df_an2)

In [ ]:
l1 = np.matmul(X2,coefs_0) + intercepts_0
l1 = relu(l1)
l2 = np.matmul(l1, coefs_1) + intercepts_1
l2 = relu(l2)
l3 = np.matmul(l2, coefs_2) + intercepts_2
l3 = softmax(l3)
pred_2 = np.argmax(l3,axis=1)-1

In [ ]:
pred_2.shape

In [ ]:
len(df_an2)

In [ ]:
df_an2.shape

In [ ]:
df_an2['SL'] = 0
df_an2['TP'] = 0 
df_an2['SIGNAL'] = 0
df_an2[df_an2.SIGNAL !=0 ]

In [ ]:
df_an2

In [ ]:
df_an2['SL'] = 0
df_an2['TP'] = 0 
df_an2['SIGNAL'] = 0
for i in range(4,len(df_an2)):
#     if df_test.pivots_h[i-1] or df_test.pivots_l[i-1]:
    if df_an2.pivots_l[i-1]==1:
        if(pred_2[i-4] == 1.0):
            df_an2.SIGNAL[i] = pred_2[i-4]
            df_an2['SL'][i] = df_an2.low[i-1] 
            df_an2['TP'][i] = 2*(df_an2.close[i] - df_an2.low[i-1]) + df_an2.close[i]
    if df_test.pivots_h[i-1]==1:
        if (pred_2[i-4]== -1.0 ):
            df_an2.SIGNAL[i] = pred_2[i-4]
            df_an2['SL'][i] = df_an2.high[i-1]
            df_an2['TP'][i] = 2*(df_an2.close[i] - df_an2.high[i-1] ) + df_an2.close[i]

In [ ]:
gt = GuruTester2(
        df_an2,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m52 = gt.run_test()

In [ ]:
(df_an2.SIGNAL !=0).sum()

In [ ]:
(df_test.SIGNAL !=0).sum()

In [29]:
with open("res_cl.txt",) as f:
    lines = f.readlines()

In [31]:
lines[0][:-1].split('\t')

['2022.08.03 13:00:00', ',', '1', '1.22121', '1.21656', '1.21811']

In [33]:
pred_mql = [int(l[:-1].split('\t')[2]) for l in lines]
tp_mql = [float(l[:-1].split('\t')[3]) for l in lines]
sl_mql = [float(l[:-1].split('\t')[4]) for l in lines]
close_mql = [float(l[:-1].split('\t')[5]) for l in lines]

In [34]:
time_mql = []
for l in lines:
    y = int(l[:-1].split('\t')[0].split(' ')[0].split('.')[0])
    m = int(l[:-1].split('\t')[0].split(' ')[0].split('.')[1])
    d = int(l[:-1].split('\t')[0].split(' ')[0].split('.')[2])
    h = int(l[:-1].split('\t')[0].split(' ')[1].split(':')[0]) 
    t = datetime(y,m,d,h)
    time_mql.append(t)
    

In [35]:
df_mql = pd.DataFrame({'time':time_mql,'pred_mql':pred_mql,'tp_mql':tp_mql,'sl_mql':sl_mql,'clsoe_mql':close_mql,})
df_mql
# df_mql["time"] = pd.to_datetime(df_an2["time"], unit="s")

,time,pred_mql,tp_mql,sl_mql,clsoe_mql
0,2022-08-03 13:00:00,1,1.22121,1.21656,1.21811
1,2022-08-03 15:00:00,-1,1.21116,1.22049,1.21738
2,2022-08-04 09:00:00,-1,1.21227,1.21596,1.21473
3,2022-08-04 15:00:00,1,1.22056,1.20649,1.21118
4,2022-08-05 05:00:00,1,1.21550,1.21295,1.21380
...,...,...,...,...,...
721,2023-03-23 10:00:00,-1,1.22750,1.23434,1.23206
722,2023-03-23 15:00:00,-1,1.22694,1.23375,1.23148
723,2023-03-24 09:00:00,1,1.22798,1.22666,1.22710
724,2023-03-24 17:00:00,-1,1.22064,1.22319,1.22234


In [36]:
merged = pd.merge(left=df_mql, right=df_test, on='time', how='left')

In [37]:
merged.dropna(inplace=True)
merged.reset_index(drop=True, inplace=True)

In [38]:
merged[merged.SIGNAL != merged.pred_mql]

,time,pred_mql,tp_mql,sl_mql,clsoe_mql,open,high,low,close,TP,SL,pivots_l,pivots_h,SIGNAL
2,2022-08-05 05:00:00,1,1.21550,1.21295,1.21380,1.21399,1.21457,1.21367,1.21380,1.2155,1.21295,0.0,0.0,0.0
66,2022-08-25 02:00:00,-1,1.17741,1.17969,1.17893,1.17919,1.17953,1.17882,1.17893,0.0000,0.00000,0.0,0.0,0.0
89,2022-08-31 23:00:00,1,1.16548,1.16056,1.16220,1.16140,1.16226,1.16116,1.16220,0.0000,0.00000,0.0,0.0,0.0
378,2022-11-24 23:00:00,-1,1.20943,1.21210,1.21121,1.21186,1.21186,1.21118,1.21121,0.0000,0.00000,0.0,0.0,0.0
379,2022-11-25 01:00:00,1,1.21209,1.21068,1.21115,1.21126,1.21173,1.21081,1.21115,0.0000,0.00000,0.0,1.0,0.0
403,2022-12-02 15:00:00,-1,1.19660,1.22936,1.21844,1.22918,1.22934,1.21339,1.21844,0.0000,0.00000,1.0,0.0,0.0
633,2023-02-24 06:00:00,-1,1.20013,1.20313,1.20213,1.20224,1.20239,1.20160,1.20213,0.0000,0.00000,1.0,0.0,0.0
642,2023-02-28 00:00:00,-1,1.20440,1.20635,1.20570,1.20459,1.20599,1.20447,1.20570,0.0000,0.00000,1.0,0.0,0.0
649,2023-03-02 12:00:00,-1,1.18584,1.20096,1.19592,1.19729,1.19823,1.19566,1.19592,0.0000,0.00000,1.0,0.0,0.0


In [60]:
(merged[merged.SL != merged.sl_mql].SL == 0).sum()

401

In [ ]:
df_1 = df_test[['time','pivots_l','pivots_h','SIGNAL']]

In [ ]:
df_1[df_2.SIGNAL != df_1.SIGNAL].SIGNAL.max()

In [ ]:
df_2[df_2.SIGNAL != df_1.SIGNAL]

In [ ]:
df_1[df_2.SIGNAL != df_1.SIGNAL]

In [ ]:
df_test[df_test.SIGNAL != 0]

In [ ]:
df_test.pivots_l[4]

In [ ]:
df_test = df_test.loc[17:]
df_test.reset_index(drop=True,inplace=True)
df_test

In [ ]:
df_an22 = df_an2.iloc[:-2]
df_an22

In [ ]:
df_test = df_test.loc[15:]
df_test

In [ ]:
df_test.reset_index(drop=True,inplace=True)

In [ ]:
df_test.drop(3948,inplace=True) 
df_test

In [ ]:
df_test[df_test.SIGNAL !=0]

In [ ]:
df_an22[]

In [ ]:
df_test[df_an22.SIGNAL!=df_test.SIGNAL]

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio = 1 #abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])

In [ ]:
rates = Rates(pair, 1000, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
# df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an = apply_patterns_in_range(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)

df_an = BollingerBandsFeature(df_an)
df_an = ATRFeature(df_an)
df_an = KeltnerChannelsFeature(df_an)
df_an = RSIFeature(df_an)
df_an = MACDFeature(df_an)

df_an['pivots_l'] = False
df_an['pivots_h'] = False

df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an['pivots_l'] = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an['pivots_h'] = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

df_an.pivots_l = df_an.pivots_l.astype(float)
df_an.pivots_h = df_an.pivots_h.astype(float)

df_an

In [ ]:
X_valid = []
Y_valid = []

indeces = []
for i in range(4,len(df_an)):
    #if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
    feature = []
    feature.append(df_an.pivots_h[i-1])
    feature.append(df_an.pivots_l[i-1])
    feature.append(df_an.pivots_h[i-2])
    feature.append(df_an.pivots_l[i-2])
    feature.append(df_an.pivots_h[i-3])
    feature.append(df_an.pivots_l[i-3])
    feature.append(df_an.pivots_h[i-4])
    feature.append(df_an.pivots_l[i-4])
    for fc in feature_col:
        feature.append(df_an[fc][i])
        feature.append(df_an[fc][i-1])
        feature.append(df_an[fc][i-2])
        feature.append(df_an[fc][i-3])
        feature.append(df_an[fc][i-4])

    X_valid.append(feature)
    indeces.append(i)

In [ ]:
predict_valid = clf.predict(X_valid)

In [ ]:
predict_valid.shape

In [ ]:
df_an.drop(0,inplace=True) 
df_an.drop(1,inplace=True) 
df_an.drop(2,inplace=True) 
df_an.drop(3,inplace=True)
df_an.drop(feature_col, axis=1, inplace=True)
df_an

In [ ]:
df_an['predict'] = predict_valid

In [ ]:
for i in range(len(df_an)):
    print(df_an['time'].iloc[i],"|\t|",predict_valid[i])

In [ ]:
l1 = np.matmul(np.array(X_valid),clf.coefs_[0])+clf.intercepts_[0]
l1 = relu(l1)

l2 = np.matmul(l1,clf.coefs_[1])+clf.intercepts_[1]
l2 = relu(l2)

l3 = np.matmul(l2,clf.coefs_[2])+clf.intercepts_[2]
l3 = softmax(l3)
l3.shape

In [ ]:
2023.03.25 23:29:55.448	nnInrange (GBPUSD,H1)	2023.03.24 20:00:00| |[[-8.804903736402808,-12.04343885613141,-6.687055078835885]]


In [ ]:
l3[-2]

In [ ]:
l3